# CSC6711 Project 5 - Content Based Filtering

* **Author**: Jacob Buysse

This notebook is an analysis of the full BoardGameGeek dataset and a comparison of MLP Regression and Random Forest Regression models using features extracted from the full dataset.

Download the datasets from [BoardGameGeek on Kaggle](https://www.kaggle.com/datasets/threnjen/board-games-database-from-boardgamegeek?resource=download)
to the `datasets` directory.


We will be using the following libraries:

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.stats import linregress
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import GroupShuffleSplit, train_test_split

Let us configure matplotlib for readable labels, high resolution, and automatic layout.

In [2]:
matplotlib.rc('axes', labelsize=16)
matplotlib.rc('figure', dpi=150, autolayout=True)

## Part 1 - Data Exploration and Joining Tables

Let us start taking a look at each one of the files in the dataset.

In [8]:
artists_df = pd.read_csv('./datasets/artists_reduced.csv')
artists_df

,Harald Lieske,Franz Vohwinkel,Peter Whitley,Scott Okumura,(Uncredited),Doris Matthäus,Alan R. Moon,Alexander Jung,Andrea Boekhoff,Björn Pertoft,...,Nathan Meunier,Andrey Gordeev,Zbigniew Umgelter,Jeppe Norsker,Daniel Profiri,Aleksander Zawada,Simon Douchy,Felix Wermke,BGGId,Low-Exp Artist
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,1
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,1
4,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21920,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,347146,0
21921,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,347521,1
21922,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,348955,0
21923,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,349131,0


21925 rows, in theory one for each board game (by BGGId) and a column for each artist (with more than 3 works), one column for `(Uncredited)`, one column for a flag of whether it included an artist with fewer than 3 works `Low-Exp Artist`.  Each entry is a boolean flag.  There are 1681 columns, so 1 ID, 1578 unique artists, 1 uncredited, and 1 low-experience artists.

In [7]:
designers_df = pd.read_csv('./datasets/designers_reduced.csv')
designers_df

,Karl-Heinz Schmiel,"G. W. ""Jerry"" D'Arcey",Reiner Knizia,Sid Sackson,Jean du Poël,Martin Wallace,Richard Ulrich,Wolfgang Kramer,Alan R. Moon,Uwe Rosenberg,...,Thomas Dupont,Mathieu Casnin,Sean Fletcher,Moritz Dressler,Molly Johnson,Robert Melvin,Shawn Stankewich,Nathan Meunier,BGGId,Low-Exp Designer
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,1
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21920,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,347146,1
21921,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,347521,0
21922,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,348955,1
21923,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,349131,1


Similar flags table with designers instead of artists (1594 columns: 1 id, 1 low-exp, and 1592 designers).

In [10]:
publishers_df = pd.read_csv('./datasets/publishers_reduced.csv')
publishers_df

,Hans im Glück,Moskito Spiele,Portal Games,Spielworxx,Stronghold Games,"Valley Games, Inc.",YOKA Games,sternenschimmermeer,E.S. Lowe,Milton Bradley,...,Cacahuete Games,BlackSands Games,Norsker Games,Perro Loko Games,Funko Games,Origame,Deep Print Games,Hidden Industries GmbH,BGGId,Low-Exp Publisher
0,1,1,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,2,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21920,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,347146,0
21921,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,347521,0
21922,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,348955,0
21923,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,349131,0


Another similar file for publishers with 1866 columns (1 ID, 1 Low-exp, 1864 publishers).

In [9]:
mechanics_df = pd.read_csv('./datasets/mechanics.csv')
mechanics_df

,BGGId,Alliances,Area Majority / Influence,Auction/Bidding,Dice Rolling,Hand Management,Simultaneous Action Selection,Trick-taking,Hexagon Grid,Once-Per-Game Abilities,...,Contracts,Passed Action Token,King of the Hill,Action Retrieval,Force Commitment,Rondel,Automatic Resource Growth,Legacy Game,Dexterity,Physical
0,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,4,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21920,347146,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
21921,347521,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21922,348955,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
21923,349131,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The mechanics dataset includes 157 different categories of board game mechanics included in the game.

In [11]:
subcategories_df = pd.read_csv('./datasets/subcategories.csv')
subcategories_df

,BGGId,Exploration,Miniatures,Territory Building,Card Game,Educational,Puzzle,Collectible Components,Word Game,Print & Play,Electronic
0,1,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,1,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0
4,5,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
21920,347146,0,0,0,1,0,0,0,0,0,0
21921,347521,0,0,0,0,0,0,0,0,0,0
21922,348955,0,0,0,0,0,0,0,0,1,0
21923,349131,0,0,0,0,0,0,0,0,0,0


These are high level categories (only 10 of them) for each game.

In [12]:
themes_df = pd.read_csv('./datasets/themes.csv')
themes_df

,BGGId,Adventure,Fantasy,Fighting,Environmental,Medical,Economic,Industry / Manufacturing,Transportation,Science Fiction,...,Theme_Fashion,Theme_Geocaching,Theme_Ecology,Theme_Chernobyl,Theme_Photography,Theme_French Foreign Legion,Theme_Cruise ships,Theme_Apache Tribes,Theme_Rivers,Theme_Flags identification
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21920,347146,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21921,347521,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21922,348955,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
21923,349131,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


These are different themes for each game (218 columns: 1 ID, 217 themes).

In [13]:
distribution_df = pd.read_csv('./datasets/ratings_distribution.csv')
distribution_df

,BGGId,0.0,0.1,0.5,1.0,1.1,1.2,1.3,1.4,1.5,...,9.2,9.3,9.4,9.5,9.6,9.7,9.8,9.9,10.0,total_ratings
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.0,5.0,11.0,86.0,3.0,4.0,6.0,8.0,426.0,5352.0
1,2,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,17.0,562.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20.0,7.0,4.0,77.0,3.0,1.0,5.0,3.0,477.0,15148.0
3,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,342.0
4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,18.0,8.0,4.0,82.0,7.0,9.0,10.0,5.0,905.0,18387.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21920,347146,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,42.0
21921,347521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,71.0
21922,348955,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,67.0
21923,349131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0


Huh.  Not sure how useful this one would be (just an aggregate of ratings for each board game) vs. computing stuff directly.  Either way - can't hurt to include extra data.  This has _almost_ one column per 1/10th rating from 0 to 10 (skips from 0.5 to 1.0) and a total_ratings column.

In [15]:
ratings_df = pd.read_csv('./datasets/user_ratings.csv')
ratings_df

,BGGId,Rating,Username
0,213788,8.0,Tonydorrf
1,213788,8.0,tachyon14k
2,213788,8.0,Ungotter
3,213788,8.0,brainlocki3
4,213788,8.0,PPMP
...,...,...,...
18942210,165521,3.0,rseater
18942211,165521,3.0,Bluefox86
18942212,165521,3.0,serginator
18942213,193488,1.0,CaptainCattan


This was the dataset we were using in our other labs.  It has the item DI, rating (0-10), and username (text).
There are a total of 18,942,215 (~19M) rows.

In [16]:
ratings_df.Username.nunique()

411374

There are 411,374 unique users that have at least one rating.

In [17]:
ratings_df.BGGId.nunique()

21925

We can also see that all 21,925 games listed in the other tables have at least one rating in this table.

In [18]:
games_df = pd.read_csv('./datasets/games.csv')
games_df

,BGGId,Name,Description,YearPublished,GameWeight,AvgRating,BayesAvgRating,StdDev,MinPlayers,MaxPlayers,...,Rank:partygames,Rank:childrensgames,Cat:Thematic,Cat:Strategy,Cat:War,Cat:Family,Cat:CGS,Cat:Abstract,Cat:Party,Cat:Childrens
0,1,Die Macher,die macher game seven sequential political rac...,1986,4.3206,7.61428,7.10363,1.579790,3,5,...,21926,21926,0,1,0,0,0,0,0,0
1,2,Dragonmaster,dragonmaster tricktaking card game base old ga...,1981,1.9630,6.64537,5.78447,1.454400,3,4,...,21926,21926,0,1,0,0,0,0,0,0
2,3,Samurai,samurai set medieval japan player compete gain...,1998,2.4859,7.45601,7.23994,1.182270,2,4,...,21926,21926,0,1,0,0,0,0,0,0
3,4,Tal der Könige,triangular box luxurious large block tal der k...,1992,2.6667,6.60006,5.67954,1.231290,2,4,...,21926,21926,0,0,0,0,0,0,0,0
4,5,Acquire,acquire player strategically invest business t...,1964,2.5031,7.33861,7.14189,1.335830,2,6,...,21926,21926,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21920,347146,Salvage,oil tanker fire rescue team send deal damage...,2021,0.0000,7.45000,5.52462,1.554930,3,4,...,21926,21926,0,0,0,0,0,0,0,0
21921,347521,Blitzkrieg!: World War Two in 20 Minutes,new square edition include nippon expansion up...,2019,1.6667,7.92276,5.56323,0.894204,1,2,...,21926,21926,0,0,0,0,0,0,0,0
21922,348955,Rock Paper Scissors: Deluxe Edition,million year people force play timehonored gam...,2021,4.5625,6.96970,5.53964,4.052570,1,3,...,21926,21926,0,0,0,0,0,0,0,0
21923,349131,Splitter,splitter group number score point mdash s s md...,2021,1.0000,6.48235,5.52631,1.160070,1,12,...,21926,21926,0,0,0,0,0,0,0,0


In [21]:
games_df.describe().T

,count,mean,std,min,25%,50%,75%,max
BGGId,21925.0,117652.663216,104628.721777,1.000000,12346.000000,105305.00000,206169.00000,349161.00000
YearPublished,21925.0,1985.494914,212.486214,-3500.000000,2001.000000,2011.00000,2017.00000,2021.00000
GameWeight,21925.0,1.982131,0.848983,0.000000,1.333300,1.96880,2.52520,5.00000
AvgRating,21925.0,6.424922,0.932477,1.041330,5.836960,6.45395,7.05245,9.91429
BayesAvgRating,21925.0,5.685673,0.365311,3.574810,5.510300,5.54654,5.67989,8.51488
StdDev,21925.0,1.516374,0.285578,0.196023,1.320720,1.47688,1.66547,4.27728
MinPlayers,21925.0,2.007343,0.693093,0.000000,2.000000,2.00000,2.00000,10.00000
MaxPlayers,21925.0,5.707868,15.014643,0.000000,4.000000,4.00000,6.00000,999.00000
ComAgeRec,16395.0,10.004391,3.269157,2.000000,8.000000,10.00000,12.00000,21.00000
LanguageEase,16034.0,216.461819,236.595136,1.000000,24.027778,138.00000,351.00000,1757.00000


Okay, this is the big one (not at straightforward as the other ones).  There is still one record for each game.
* BGGId - ID
* Name - Text name of the game (might be able to construct some manual features, # words in title, length of title)
* Description - All lowercase, stripped of punctuation, and lemmatized (should be good for bad of words)
* YearPublished - Year published.  Though it looks like at least one zero in there, and the min is -3500?!
* GameWeight - Looks like a gut feel for how "heavy" a game is, mechanics-wise, from 0 to 5.
* AvgRating - Probably easily computed (should be sum(ratings) / count(ratings) - or possibly "- 1" for "sampled average" (0-10)
* BayesAvgRating - Weighted average rating taking into account reviews (0-10).
* StdDev - Standard deviation of the "Bayes" average rating.
* MinPlayers - Minimum number of players.  Though it appears to range from 0 (!?) to 10.
* MaxPlayers - Maximum number of players.  Also with a strange range of 0 (!?) to 999 (seems arbitrary).
* ComAgeRec - Community's minimum age recommendation (2-21).
* LanguageEase - Language requirement?  (1-1757).  Seems like a language complexity rating.
* BestPlayers - Community voted best number of players. (0-15).  Probably pretty sparse.
* GoodPlayers - An array of "good" player counts.  Might be able to one hot encode this based on uniqueness.
* NumOwned/NumWant/NumWish - Number of players that own, want, or wishlisted this game.
* NumWeightVotes - ? (0 - 7673).  Some count of users regarding "weight votes"?
* MfgPlaytime - Manufacturer stated play time from 0 to 60000 and avg of 90.  Probably in minutes.
* ComMinPlaytime/ComMaxPlaytime - Min/max play time provided by the community.  Also 0-60000 w/ avg 60-90.  Probably minutes.
* MfgAgeRec - Manufacturer minimum age recommendation. 0-25, avg 9.
* NumUserRatings - Number of user ratings.
* NumComments - Number of comments.  Could be an additional indicator of popularity.
* NumAlternates - Number of alternate versions of the game.
* NumExpansions - Number of expansions to the game.  Apparently not listed separately.
* NumImplementations - Curious - not sure what alternate implementations would be.
* IsReimplementation - Bit - whether this game is a re-implementation.
* Family - Internal classification (like a category).  Text.  E.g. "Classic Line (Valley Games)"
* Kickstarted - Bit - whether this was a kickstarter game.  Could indicate demand.
* ImagePath - URL for image for game.  Interesting, I wonder if I could download all the images and run them through 2D CNNs for image vectorization.
* Rank:... - Rank within 9 categories: boardgame,strategygames,abstracts,familygames,thematic,cgs,wargames,childrensgames
* Cat:... - Bit flat for whether the game is in one of 9 categories: Thematic,Strategy,War,Family,CGS,Abstract,Party,Childrens

Overall, quite a lot of data for each board game to start with.

In [26]:
game_count = 21925
user_count = 411374
rating_count = 18942215
possible_rating_count = game_count * user_count
rating_density = rating_count / possible_rating_count
print(f"Rating Density {rating_density:.2%}")

Rating Density 0.21%


### Dataset Summary

There are 9 datasets in total.  The primary datasets are the games (21,925) and the rankings (18,942,215 ratings over 411,374 unique users, a 0.21% density). There is one "rating distribution" summary dataset.  There are 3 artist/designer/publisher datasets.  And there are 3 ancillary information for board game datasets: mechanics, themes, subcategories.

In [42]:
### Joining the data

gdf = games_df\
    .merge(distribution_df, on='BGGId')\
    .merge(mechanics_df, on='BGGId')\
    .merge(themes_df, on='BGGId')\
    .merge(subcategories_df, on='BGGId')\
    .merge(artists_df, on='BGGId')\
    .merge(designers_df, on='BGGId')\
    .merge(publishers_df, on='BGGId')

BGGId
Name
Description
YearPublished
GameWeight
AvgRating
BayesAvgRating
StdDev
MinPlayers
MaxPlayers
ComAgeRec
LanguageEase
BestPlayers
GoodPlayers
NumOwned
NumWant
NumWish
NumWeightVotes
MfgPlaytime
ComMinPlaytime
ComMaxPlaytime
MfgAgeRec
NumUserRatings
NumComments
NumAlternates
NumExpansions
NumImplementations
IsReimplementation
Family
Kickstarted
ImagePath
Rank:boardgame
Rank:strategygames
Rank:abstracts
Rank:familygames
Rank:thematic
Rank:cgs
Rank:wargames
Rank:partygames
Rank:childrensgames
Cat:Thematic
Cat:Strategy
Cat:War
Cat:Family
Cat:CGS
Cat:Abstract
Cat:Party
Cat:Childrens
0.0
0.1
0.5
1.0
1.1
1.2
1.3
1.4
1.5
1.6
1.7
1.8
1.9
2.0
2.1
2.2
2.3
2.4
2.5
2.6
2.7
2.8
2.9
3.0
3.1
3.2
3.3
3.4
3.5
3.6
3.7
3.8
3.9
4.0
4.1
4.2
4.3
4.4
4.5
4.6
4.7
4.8
4.9
5.0
5.1
5.2
5.3
5.4
5.5
5.6
5.7
5.8
5.9
6.0
6.1
6.2
6.3
6.4
6.5
6.6
6.7
6.8
6.9
7.0
7.1
7.2
7.3
7.4
7.5
7.6
7.7
7.8
7.9
8.0
8.1
8.2
8.3
8.4
8.5
8.6
8.7
8.8
8.9
9.0
9.1
9.2
9.3
9.4
9.5
9.6
9.7
9.8
9.9
10.0
total_ratings
Alliances
Area Ma

In [56]:
gdf.columns.shape[0]

5665

We have more than 5k game attributes to use when computing user features.

## Data Cleaning

We know we need to to split our rating data into a training and testing dataset (75/25 split by user).  Each user will only be one of the datasets and the split percentage will be reflected in both the total number of ratings and number of users (pretty nifty).  Then, we will split the training dataset so each users ratings will be split between a seen and unseen dataset.  This means each user needs at minimum 2 ratings in order to qualify for the testing dataset.

In [53]:
print(f"Original ratings: {ratings_df.shape[0]:,}")
username_encoder = LabelEncoder()
username_encoder.fit(ratings_df.Username)
print(f"Total number of original users: {username_encoder.classes_.size:,}")
ratings_df['username_id'] = username_encoder.transform(ratings_df.Username)
counts_df = ratings_df.groupby('username_id')[['Rating']].count()
merged_df = ratings_df.merge(counts_df, on='username_id', suffixes=['', '_count'])
filtered_df = merged_df[merged_df.Rating_count > 1]
multi_df = filtered_df.copy()
print(f"Filtering out users with only one rating: {multi_df.shape[0]:,}")
print(f"Number of users with more than one rating: {multi_df.username_id.nunique():,}")

Original ratings: 18,942,215
Total number of original users: 411,375
Filtering out users with only one rating: 18,862,919
Number of users with more than one rating: 332,079


## Train/Test and Seen/Unseen split
Let us split our datasets now.  We will compute the user features after the splits are complete.

In [54]:
gss = GroupShuffleSplit(n_splits=1, train_size=0.75, random_state=777)
train_index, test_index = next(gss.split(X=multi_df, y=multi_df.Rating, groups=multi_df.username_id))
train_df = multi_df.iloc[train_index].copy()
test_df = multi_df.iloc[test_index].copy()
print(f"Train {train_df.shape[0]:,}, Test {test_df.shape[0]:,}")
print(f"Train users {train_df.username_id.nunique():,}, Test users {test_df.username_id.nunique():,}")

Train 14,119,520, Test 4,743,399
Train users 249,059, Test users 83,020


In [55]:
seen_df, unseen_df = train_test_split(test_df, train_size=0.75, random_state=777, stratify=test_df.username_id)
print(f"Seen {seen_df.shape[0]:,}, Unseen {unseen_df.shape[0]:,}")

Seen 3,557,549, Unseen 1,185,850
